In [1]:

import pandas as pd
from data.random_data import RandomOHLCV
from data.ohlcv import ServeNewOHLCV


ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = 10_000, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-50, 60),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

ohlcv

serv = ServeNewOHLCV(ohlcv)
# serv.serv_period(date='01-01-2024', start_time='09:30:00', end_time='16:00:00')
# serv.serv_period(days_ago=3, months_ago=0, start_time='08:30:00', end_time='18:00:00')
start_data = serv.serv_range(dayrange=(-5,-2), start_time='08:30:00', end_time='18:00:00')
start_data


,open,high,low,close,volume
date,,,,,
2024-04-09 08:45:00,94.02,94.38,93.94,94.15,2226
2024-04-09 09:00:00,93.97,94.40,93.88,94.33,50000
2024-04-09 09:15:00,94.21,94.72,94.06,94.44,500
2024-04-09 09:30:00,94.42,94.90,94.27,94.68,500
2024-04-09 09:45:00,94.79,94.82,94.54,94.73,21102
...,...,...,...,...,...
2024-04-12 17:00:00,95.20,95.64,95.16,95.42,3402
2024-04-12 17:15:00,95.42,95.46,95.24,95.34,50000
2024-04-12 17:30:00,95.57,95.61,95.20,95.30,500


In [2]:
serv.next_bar(histBars=1)

,open,high,low,close,volume
date,,,,,
2024-04-12 18:00:00,94.99,95.20,94.54,94.68,500
2024-04-12 18:15:00,94.96,94.97,94.69,94.74,22366


In [3]:
# serv.serv_range(dayrange=(2,8), start_time='08:30:00', end_time='18:00:00')

In [4]:
from frame.frame import Frame
f = Frame('TSLA')

#* KEEP RUNNING THIS CELL TO SEE THE CHART UPDATE

serv.next_bar(histBars=1)
f.load_ohlcv(serv.current_data)
f.setup_chart() 
f.plot(trading_hours=True)

In [5]:

from strategies.ta import MA, MACD

f.add_ta(MA('close', 9),  {'dash': 'solid', 'color': 'red', 'width': 2}) # creates new column to the dataframe with the moving average eg MA_C9
f.add_ta(MA('close', 21), {'dash': 'solid', 'color': 'blue', 'width': 2})
f.add_ta(MA('close', 28), {'dash': 'solid', 'color': 'green', 'width': 2})
f.add_ta(MACD(fast=12, slow=26, signal=9) , 
         [{'dash': 'solid', 'color': 'purple', 'width': 2}, # MACD
          {'dash': 'solid', 'color': 'pink', 'width': 2},   # signal
          {'color': 'black'}], 'macd', 2) # histogram

f.update_ta_data()
f.plot(trading_hours=True)

In [6]:

from dataclasses import dataclass

@dataclass
class Strategy:
    data_serv: ServeNewOHLCV

    def __post_init__(self):
        self.f = Frame('TSLA')
        self.f.load_ohlcv(self.data)
        self.f.setup_chart()
        self.f.add_ta(MA('close', 9),  {'dash': 'solid', 'color': 'red', 'width': 2})

In [7]:
f.load_ohlcv(serv.next_bar(histBars=1))
f.data.tail(12)

                      open   high    low  close  volume
date                                                   
2024-04-12 18:30:00  94.39  94.50  93.86  94.10     500
2024-04-12 18:45:00  94.00  94.51  94.00  94.36     500


,open,high,low,close,volume,MA_cl_9,MA_cl_21,MA_cl_28,MACD_cl_12_26_9_MACD,MACD_cl_12_26_9_Signal,MACD_cl_12_26_9_Histogram
date,,,,,,,,,,,
2024-04-12 16:00:00,94.59,95.08,94.38,94.92,500,94.475556,94.633810,94.935357,-0.195849,-0.281537,0.085688
2024-04-12 16:15:00,95.02,95.29,94.97,95.10,1584,94.636667,94.617143,94.912143,-0.140722,-0.253374,0.112652
2024-04-12 16:30:00,94.95,95.17,94.90,94.98,50000,94.721111,94.615238,94.878929,-0.105501,-0.223799,0.118299
2024-04-12 16:45:00,94.66,95.23,94.51,94.96,500,94.822222,94.595714,94.839286,-0.078299,-0.194699,0.116401
2024-04-12 17:00:00,95.20,95.64,95.16,95.42,3402,94.897778,94.595714,94.828571,-0.019399,-0.159639,0.140240
2024-04-12 17:15:00,95.42,95.46,95.24,95.34,50000,94.950000,94.600000,94.798929,0.020586,-0.123594,0.144181
2024-04-12 17:30:00,95.57,95.61,95.20,95.30,500,94.998889,94.620000,94.786071,0.048489,-0.089178,0.137666
2024-04-12 17:45:00,95.09,95.46,94.99,95.36,500,95.077778,94.633810,94.777500,0.074583,-0.056425,0.131009
2024-04-12 18:00:00,94.99,95.20,94.54,94.68,500,95.117778,94.649048,94.750000,0.039933,-0.037154,0.077087


# Putting it all together

In [1]:
import pandas as pd
from data.random_data import RandomOHLCV
from data.ohlcv import ServeNewOHLCV
from strategies.ta import MA, MACD, HPLP, SupRes, ATR
from strategies.signals import Tail, PullbackNear, Overlap
from frame.frame import Frame


ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = 10_000, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-50, 60),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

serv = ServeNewOHLCV(ohlcv)
start_data = serv.serv_range(dayrange=(-5,-2), start_time='08:30:00', end_time='18:00:00') # used as a starting point for the backtest


f = Frame('TSLA')
f.add_ta(ATR(span=50), {}, chart_type=None)
f.add_ta(MA('close', 9),  {'dash': 'solid', 'color': 'magenta', 'width': 2}) # appends ta to the ta list
f.add_ta(MA('close', 21), {'dash': 'solid', 'color': 'blue', 'width': 2})
f.add_ta(MA('close', 28), {'dash': 'solid', 'color': 'green', 'width': 2})
f.add_ta(MACD(fast=12, slow=26, signal=9) , 
         [{'dash': 'solid', 'color': 'purple', 'width': 2}, # MACD
          {'dash': 'solid', 'color': 'pink', 'width': 2},   # signal
          {'color': 'black'}], 'macd', 2) # histogram

f.add_ta(HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, # high
          {'color': 'red', 'size': 10}], 
          chart_type = 'points')

f.add_ta(SupRes(hi_point_col='HP_hi_10', lo_point_col='LP_lo_10', atr_col='ATR_50', tolerance=1),
            [{'dash': 'solid', 'color': 'green', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'dash': 'solid', 'color': 'red', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'support_resistance') 

f.add_signals(Tail(normRange=(0,100), tailExceedsNthBarsAgo=2), 
              {'dash': 'solid', 'color': 'red', 'width': 2}, 'lines+markers', row=4) # tail signal

f.add_signals(PullbackNear(maxCol='HP_hi_10', minCol='LP_lo_10', normRange=(0,100), longPullbackCol='MA_cl_21', shortPullbackCol='MA_cl_21'),
              {'dash': 'solid', 'color': 'blue', 'width': 2}, 'lines+markers', row=4) # pullback signal

f.add_signals(Overlap(normRange=(0,100)),
              {'dash': 'solid', 'color': 'purple', 'width': 2}, 'lines+markers', row=4) # overlap signal


serv.next_bar(histBars=1) 
f.load_ohlcv(serv.current_data)
f.setup_chart() 
f.update_ta_data()
f.update_signals_data()
f.plot(trading_hours=True, height=1200, width=2000)


# backtest = BacktestFrames(f, serv, strategies=[strat1], barsToRun=20, stopAtEndOfDay=True)

"""
Resistance: 

The objective is to modify a support and resistance class that will find the support and resistance levels and return a pandas data frame with those levels projected forward to the end of the data frame from the points that have been detected
So it is important that every row of the data frame is filled in with the same price point stretching from the index where the point is located to the end of the data frame so that it displays correctly on a chart. (This is not a visualisation task but a data manipulation task)
Do not worry about tolerances and touch points for now.

  -- All code snippets are to be returned in the chat do not update files directly
  -- compute only base on the last close price on the last bar
  -- hp and lp is high point or low point 
  -- HP and LP are already computed and should be obtained from the data frame by referencing the relevant columns
  -- find the next  hp or lp That is higher than the close price. I suggest putting the mall in a list both high points and low points and sorting them in ascending order first.
  -- When finding the next higher point use the argument pointsAgo to find the point that is x points higher from the current point.
  -- now find to index Of the point that was found whether it was a low point or a high point is irrelevant. 
  -- Now project the point that was found and forward fill it to the end of the df So that when visualised on a chart there will be a line stretching from the point that has been found to the end of the chart. 
  
  Support:
  -- Similar to resistance but of course the opposite.

  """

# backtest.run()

'\nResistance: \n\nThe objective is to modify a support and resistance class that will find the support and resistance levels and return a pandas data frame with those levels projected forward to the end of the data frame from the points that have been detected\nSo it is important that every row of the data frame is filled in with the same price point stretching from the index where the point is located to the end of the data frame so that it displays correctly on a chart. (This is not a visualisation task but a data manipulation task)\nDo not worry about tolerances and touch points for now.\n\n  -- All code snippets are to be returned in the chat do not update files directly\n  -- compute only base on the last close price on the last bar\n  -- hp and lp is high point or low point \n  -- HP and LP are already computed and should be obtained from the data frame by referencing the relevant columns\n  -- find the next  hp or lp That is higher than the close price. I suggest putting the ma

In [3]:
f.data

,open,high,low,close,volume,ATR_50,MA_cl_9,MA_cl_21,MA_cl_28,MACD_cl_12_26_9_MACD,...,Sup_1_Lower,Sup_2,Sup_2_Upper,Sup_2_Lower,SigL_Tail,SigS_Tail,SigL_PBN,SigS_PBN,SigL_Olap,SigS_Olap
date,,,,,,,,,,,,,,,,,,,,,
2024-04-09 08:45:00,115.62,116.22,115.46,115.98,50000,0.760000,NaN,NaN,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-09 09:00:00,116.17,116.29,115.50,115.81,500,0.775000,NaN,NaN,NaN,-0.013561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-09 09:15:00,116.12,116.53,115.82,116.29,500,0.753333,NaN,NaN,NaN,0.014259,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-09 09:30:00,116.18,116.42,116.04,116.40,500,0.660000,NaN,NaN,NaN,0.044668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-09 09:45:00,116.60,116.80,116.25,116.40,500,0.638000,NaN,NaN,NaN,0.067983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-12 17:15:00,125.10,125.86,124.93,125.59,29057,0.627800,125.210000,124.403810,123.993929,0.567521,...,124.06,123.18,123.18,122.76,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-12 17:30:00,125.45,125.47,125.33,125.33,500,0.621600,125.103333,124.513810,124.054643,0.565493,...,124.06,123.18,123.18,122.76,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-12 17:45:00,125.15,125.45,124.51,124.67,30487,0.626200,125.013333,124.590952,124.119643,0.504811,...,124.06,123.18,123.18,122.76,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
import numpy as np

d= {'L_Tail': np.float64(39.81), 'S_Tail': 0, 'L_PBN': 0, 'S_PBN': 32.9, 'L_Olap': 0, 'S_Olap': 0}

df = pd.DataFrame(d, index=[f.data.index[-1]])
df

,L_Tail,S_Tail,L_PBN,S_PBN,L_Olap,S_Olap
2024-04-12 18:15:00,39.81,0,0,32.9,0,0


In [9]:
f.sigs[0][0].run('LONG', f.data.iloc[:-23])
f.sigs[1][0].run('SHORT', f.data.iloc[:-20])

0

In [10]:
style  = {'dash': 'solid', 'color': 'green', 'fillcolour': 'rgb(0, 255, 61, 0.1)', 'width': 2}
style  = {'dash': 'solid', 'color': 'green',  'width': 2}
fillcolour = style.get('fillcolour', 'rgba(0, 0, 255, 0.1)')

fillcolour

'rgba(0, 0, 255, 0.1)'

In [11]:
f.data.tail(12).columns

Index(['open', 'high', 'low', 'close', 'volume', 'ATR_50', 'MA_cl_9',
       'MA_cl_21', 'MA_cl_28', 'MACD_cl_12_26_9_MACD',
       'MACD_cl_12_26_9_Signal', 'MACD_cl_12_26_9_Histogram', 'HP_hi_10',
       'LP_lo_10', 'Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2',
       'Res_2_Upper', 'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower',
       'Sup_2', 'Sup_2_Upper', 'Sup_2_Lower'],
      dtype='object')

In [12]:
# create a df with random hp and lp points and random intervals between them. the points should not be on every row.  the y only occure evry so often  and the intervals between them are random. us np.nan to fill in the gaps

import pandas as pd
from data.random_data import RandomOHLCV
from data.ohlcv import ServeNewOHLCV
from strategies.ta import MA, MACD, HPLP, SupRes
from frame.frame import Frame


ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = 10_000, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-50, 60),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

serv = ServeNewOHLCV(ohlcv)
start_data = serv.serv_range(dayrange=(-5,-2), start_time='08:30:00', end_time='18:00:00') # used as a starting point for the backtest
f = Frame('TSLA')

f.add_ta(HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, # high
          {'color': 'red', 'size': 10}], 
          chart_type = 'points')
serv.next_bar(histBars=1) 
f.load_ohlcv(serv.current_data)
f.setup_chart() 
f.update_ta_data()
f.plot( height=800, width=1200)

hplp = f.data[['close', 'HP_hi_10', 'LP_lo_10']].copy() # ['close', 'HP', 'LP']


In [13]:

# 1. get close price from the last bar 
# 2. get all high points and low points in alist and sort them in ascending order
# 3. if the last bar is green ie bullish then finf resistnce if bearish find support
# 4. assuming a last bullish bar  find the next point that is higher than the close price
# 5. find the upper and lower bounds by seeing the max and min of the high and low points within a renge as determineed by a tolerance value which then computes the ART multiple of the range
# log resistance and its bounds in a dictionary
# now do the same agians for the next resisatne level using the last resistance level upperbounds (plus a tolerance factor) as the starting point instead fo the close price
#  now do the same for the support levels but in the opposite direction but start with the min of the  lowerbounds of the low values and the lowerbouns of the res  as the starting point

hplp.rename(columns={'HP_hi_10': 'HP', 'LP_lo_10': 'LP'}, inplace=True)


# 1. get close price from the last bar 
close_price = f.data['close'].iloc[-1]

# 2. get all high points and low points in alist and sort them in ascending order
hp = hplp['HP'].dropna().to_list()
lp = hplp['LP'].dropna().to_list()
hp_lp = hp + lp
print(f'{hp_lp=}')
print('------')


# 3. if the last bar is green ie bullish then finf resistnce if bearish find support
last_close = f.data['close'].iloc[-1]
last_open = f.data['open'].iloc[-1]
last_color = 'green' if last_close > last_open else 'red'
print(f'{last_color=}')
print(f'{last_close=}')
print('------')

# 4. function to find the next higher or lower val in a list 
def find_next_val(val, lst, pointsAgo=1, updn='up'):
    lst = sorted(lst)
    if   updn == 'up': next_val = [x for x in lst if x > val][pointsAgo]
    elif updn == 'dn': next_val = [x for x in lst if x < val][pointsAgo]
    return next_val

next_hp = find_next_val(last_close, hp_lp, 1) 
next_lp = find_next_val(last_close, hp_lp, 1, 'dn')
print(f'{next_hp=}')
print(f'{next_lp=}')
print('------')

# 5. find the upper and lower bounds by seeing the max and min of the high and low points within a renge as determineed by a tolerance value which then computes the ART multiple of the range
def find_bounds(val, lst, tolerance=0.5):
    lst = sorted(lst)
    min_val = [x for x in lst if x < val][-1]
    max_val = [x for x in lst if x > val][0]
    rng = max_val - min_val
    ART = rng * tolerance
    return min_val - ART, max_val + ART

tolerance = 0.5
min_res, max_res = find_bounds(next_hp, hp_lp, tolerance=0.01)
print(f'{min_res=}')
print(f'{max_res=}')
print('------')

# log resistance and its bounds in a dictionary
resistance = {'lower': min_res, 'res': next_hp, 'upper': max_res}
print(f'{resistance=}')
print('------')

hplp

hp_lp=[85.75, 86.42, 85.44, 84.19, 81.18, 81.03, 79.49, 79.49, 78.6, 78.09, 84.55, 84.1, 78.95, 79.74, 78.33, 76.89, 76.31, 76.46]
------
last_color='green'
last_close=np.float64(77.13)
------
next_hp=78.33
next_lp=76.46
------
min_res=78.0849
max_res=78.6051
------
resistance={'lower': 78.0849, 'res': 78.33, 'upper': 78.6051}
------


,close,HP,LP
date,,,
2024-04-09 08:45:00,85.34,NaN,NaN
2024-04-09 09:00:00,85.38,NaN,NaN
2024-04-09 09:15:00,85.47,85.75,NaN
2024-04-09 09:30:00,85.53,NaN,NaN
2024-04-09 09:45:00,85.28,NaN,NaN
...,...,...,...
2024-04-12 17:15:00,76.96,NaN,NaN
2024-04-12 17:30:00,76.61,NaN,76.46
2024-04-12 17:45:00,77.06,NaN,NaN
